In [30]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


    

chat = ChatOpenAI(temperature=0.1,
                   streaming=True, 
                   callbacks=[
                      StreamingStdOutCallbackHandler(),
                    ],
                  )

a_template = PromptTemplate.from_template("What is capital of {country}")
b_template = PromptTemplate(
  template="What is capital of {country}",
  input_variables=["country"],
)
a_template.format(country="Austria")
b_template.format(country="Austrailia")
# result = t | chat
# result.invoke({"country":"Austria"})

'What is capital of Austrailia'

In [32]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

# prompt.format(country="Turkey")
chain = prompt |chat
chain.invoke({
  "country":"Turkey"
})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira')

In [ ]:


example_prompt_chat = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
])

prompt_chat = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt_chat,
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages(
  [
  ("system","You are a geography expert"),
  prompt_chat,
  ("human", "What do you know about {country}?"),
])

chain = final_prompt |chat
chain.invoke({"country":"Israel"})


In [38]:

class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables) :
    from random import choice
    return [choice(self.examples)]
  
  
example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=380,
)

# 내가 만든거
example_selector_my = RandomExampleSelector(  
  examples = examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector_my,

  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

prompt.format(country="Brazil")

'Human:What do you know about Italy? \n AI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman:What do you know about Brazil?'

In [46]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt

prompt=load_prompt("./prompt.yaml")

chat = ChatOpenAI(temperature=0.1,
                   streaming=True, 
                   callbacks=[
                      StreamingStdOutCallbackHandler(),
                    ],
                  )
prompt.format(country="Germany")

'What is the capital of Germany'

In [57]:
from langchain.prompts.pipeline import PipelinePromptTemplate



intro= PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example= PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start= PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final= PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts= [
    ("intro", intro),
    ("example", example),
    ("start", start),
]




full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts,)
# full_prompt.format(
#   character ="Pirate",
#   example_question="What is your location?",
#   example_answer="Arrrg! That is a secret!!  Arg Arg!!",
#   question="What is your food?",
# )

chain = full_prompt | chat
chain.invoke({
    "character": "Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret!!  Arg Arg!!",
    "question":"What is your food?",
})

Arrrg! Me favorite food be rum and fish, matey! Aye, nothing beats a good ol' feast of fresh fish and a barrel of rum to wash it down with! Arg Arg!

AIMessageChunk(content="Arrrg! Me favorite food be rum and fish, matey! Aye, nothing beats a good ol' feast of fresh fish and a barrel of rum to wash it down with! Arg Arg!")

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

#모든 response가 메모리에 저장된다.

# set_llm_cache(InMemoryCache()) 
# set_llm_cache(SQLiteCache("cache.db")) 
set_debug(True)

chat=ChatOpenAI(
  temperature=0.1,

)
chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a basic recipe for making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salte

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a basic recipe for making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [ ]:
chat.predict("How do you make italian pasta?")


In [67]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback


chat=ChatOpenAI(
  temperature=0.1, 
)

with get_openai_callback() as usage:

  a=chat.predict("What is the recipe for soju")
  b=chat.predict("What is the recipe for Croissant bread")
  print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of yeast\n\nInstructions:\n1. Rinse the rice under cold water until the water runs clear.\n2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.\n3. Remove the pot from heat and let it cool to room temperature.\n4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.\n5. Add the paste to the cooled rice and mix well.\n6. Cover the pot with a clean cloth and let it ferment in a warm, dark place for 3-4 days.\n7. After fermentation, strain the mixture through a cheesecloth to remove any solids.\n8. Transfer the liquid t

In [71]:
from langchain.llms.openai import OpenAI
chat=OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")
chat.save("model.json")

In [74]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.loading import load_llm

load_llm("model.json")



/Users/moonk/Documents/coder/Python/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/moonk/Documents/coder/Python/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})